In [10]:
import pm4py
import os


In [19]:
# Load the .xes file
log = pm4py.read_xes("./dataset/edited_hh102_labour.xes")
pd = pm4py.convert_to_dataframe(log)

selected_columns = ['concept:name', 'lifecycle:transition', 'time:timestamp']
pd_subset = pd[selected_columns]


pd_subset.to_csv("./cleaned-dataset/output.csv", index=False)


parsing log, completed traces ::   0%|          | 0/18 [00:00<?, ?it/s]

In [11]:
print(os.getcwd())


C:\Users\Brend\Downloads\AI-Personal-Assistant-Scheduler
